# **Submitting scripts as batch jobs using SLURM**

Up until now, we have been working on the Hopper cluster in one of two ways:
- Connecting to the **login** node and running commands and scripts from it
- Requesting an **interactive node** allocation and running code in Jupyter Notebooks (or from the shell!)

This has served us well so far because we have been running relatively small *jobs*. A *job* is the term used for any task you command some processors to perform. In the **On Demand** server we use to run JupyterLab, our code runs within its own *job* and is assigned its own *node*. A *node* is a computing unit consisting of *cores*. Each *node* in Hopper (and in every other cluster) contains several *cores*. When we send our tasks to the compute *nodes*, they can get broken up into pieces that each *core* then handles. This requires that we write our code using a framework for splitting up our jobs into these discrete pieces. Thankfully, we've already learned about one such tool, named **Dask**. The process by which **Dask** breaks our computing work into pieces is called parallelization. 

### Sharing and allocating compute resources: Enter SLURM

Hopper is a shared resource, and there are many users with access to them.  In fact, any faculty or student at GMU can request an account! When there are so many possible users, each submitting *jobs* of varying complexity and resource needs, we need a way to fairly and efficiently allocate compute nodes and keep track of who needs what. When more resources are being requested than there are available (a more common occurrence than you might think!), we also need a way to keep track of who is requesting which resources and have a strategy to allocate them. SLURM (or Simple Utility for Resource Management) is a software created for this purpose ([named after the soft drink in the early 2000s cartoon, Futurama](https://futurama.fandom.com/wiki/Slurm)). 

The Hopper admins have given us a sample script that we can modify. They also provide documentation on SLURM in the link below:

https://wiki.orc.gmu.edu/mkdocs/Getting_Started_with_SLURM/

We can submit jobs to the compute nodes via SLURM in two ways:
1. Directly from the command line using the ```srun``` command
2. Using a batch script with the ```sbatch``` command.

### Submitting jobs from the command line with `srun`
Let's try submitting a job from the command line. Create a new file using your text editor of choice with the following code:





In [5]:
line1 = 'Climate data is a great course\n'
line2 = 'Profs Dirmeyer and Ortiz are such great instructors!'

# Open a text file in "writing" mode and write our two lines
with open('textfile.txt', 'w') as fout:
    fout.writelines([line1, line2])


Save to a file called `write_file.py` and run the following command from the shell.

`srun --ntasks=1 --cpus-per-task=1 --output=/scratch/lortizur/textfile.out python ./write_file.py &`

Depending on how busy Hopper is, our job may run immediately or we may have to wait until resources become available. Note that in our `srun` command, we specified various options, each with their own argument. They are:

`ntasks:` How many tasks we are splitting out on.

`cpus-per-task: `How many cores are we running our tasks run.

`output: `Script messages will be written to the file named textfile.out

There are options to specify many other resource requests like how much memory per task we want, how long do we want our job to run for, and which *partition* do we want it to run on. *Partitions* refer to sets of resources that may be allocated for specific purposes. for example, one partition in Hopper is singled out for users who are running jobs that use GPUs rather than regular processors (or CPUs). Other *partitions* may be created for specific needs, like running an important weather forecast every day. In hopper, we all use the default *partition*, named *normal*.

Once we submit our job, we can view its status with th `squeue` command:
`squeue -u lortizur`

### Submitting jobs using a batch script
Submitting jobs via a batch script is very similar to using a the `srun` command. Rather than writing all our resource requests in the command file, we create a file defining everything we want to run our task. (go to Hopper Slurm Doc). 

Let's go back to our lesson on grabbing data from Google Cloud. 

Often, as climate scientists, we need to work with multi-model ensembles of various models across a variety of emissions scenarios. By including various modeling groups' formulations of global circulation and other physical processes, we are able to assess a range of possible futures in the incidence of atmospheric hazards. For this excercise we want to grab as many models as we can and compute for each year in each model, how many days exceeded 40 degrees Celsius throughout the 21st century. Let's see what that looks like when we query the cloud:

In [6]:
import gc

import intake
import xarray as xr
import xesmf as xe
import numpy as np
import xclim

from dask.diagnostics import ProgressBar

xclim.set_options(data_validation='warn')


In [7]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [9]:
# Using the CMIP6 ensemble stored in google cloud by Pangeo project and opening the data store object
col_url = 'https://storage.googleapis.com/cmip6/pangeo-cmip6.json'
col = intake.open_esm_datastore(col_url)

# We are interested in four Shared Socioeconomic Pathways (SSPs) and the historical run.
scenarios = ['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']


In [10]:
# Create dict to hold the extracted dataset objects for each scenario.
print('Loading ensemble...')
dset_dict = {}
for sce in scenarios:

    col_subset = col.search(experiment_id=sce,
                            variable_id='tasmax',
                            member_id='r1i1p1f1',
                            table_id='day',
                            )
    # Convert to xarray dataset objects
    dset_dict[sce] = col_subset.to_dataset_dict(zarr_kwargs={'consolidated': True})

print('Ensemble loaded into dict!')


Loading ensemble...

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/home/lortizur/.local/lib/python3.10/site-packages/xarray/coding/times.py:682: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/lortizur/.local/lib/python3.10/site-packages/xarray/coding/times.py:682: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/lortizur/.local/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/home/lortizur/.local/lib/python3.10/site-packages/xarray/coding/times.py:6


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/home/lortizur/.local/lib/python3.10/site-packages/xarray/coding/times.py:682: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/lortizur/.local/lib/python3.10/site-packages/xarray/coding/times.py:682: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/lortizur/.local/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/home/lortizur/.local/lib/python3.10/site-packages/xarray/coding/times.py:6

Ensemble loaded into dict!


In [13]:
dset_dict['historical']['CMIP.AWI.AWI-CM-1-1-MR.historical.day.gn']['tasmax']

,Array,Chunk
Bytes,16.55 GiB,124.31 MiB
Shape,"(1, 60265, 192, 384)","(1, 442, 192, 384)"
Count,3 Graph Layers,137 Chunks
Type,float32,numpy.ndarray


In [15]:
for sce in scenarios:
    print(sce, len(dset_dict[sce].keys()))

historical 33
ssp126 23
ssp245 26
ssp370 23
ssp585 28


From this information, we know that for each of our ensemble members, we can expect several GB of memory usage, and each scenario has at least 23 such members and as many as 33! If we were to perform operations over this entire dataset without allocating the correct resources, the system would likely stop our *job* as soon as it figures out we want more than it can give. We need to specifically request the resources we need and submit our job to the *compute nodes*. 

Copy the following files into your directory:

`/home/lortizur/clim680/slurm_lesson/run_precip.py`

`/home/lortizur/clim680/slurm_lesson/precip.slurm`

`run_precip.py` is a python script that take data from the same CMIP6 ensemble as above, and computes each year's daily maximum precipitation. `precip.slurm` is our batch script that selects a set of resources to run our script.